### xgboost model to predict mean player outcomes
used in conjunction with nn algs to determine mean wrc for player. Three diffrent models based on available data then combinded to create an xgb predection.

In [1]:
import pandas as pd
import os
import numpy as np

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/')

In [2]:
# import data
data_02_24 = pd.read_csv('./data/cleaned/data_02_24.csv')
df = data_02_24.copy() # import data
df = df.drop(columns=['Unnamed: 0'])

#### find the z_score of each stat
weighted avg of year, std of year, 10 away from 100 is one std


In [3]:
def mean_year(group):
        avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
        return avg

In [4]:
# weighted std to account for low PA outlier preformances
def calculate_weighted_std(group):
    avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    variance = np.average((group - avg)**2, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    return np.sqrt(variance)

In [5]:
def z_scores(player_df, mean_df, std_df):
    # z scores for each stat and player, also creating a new df flow, add back names based on MLBAMID later
    z_scores = pd.DataFrame(index=player_df.index, columns=player_df.select_dtypes(include=[np.number]).columns)

    # scaled for each col
    for col in player_df.select_dtypes(include=[np.number]).columns:
        if col not in ['Season', 'MLBAMID']:  # not needed
            for idx in player_df.index:
                season = player_df.loc[idx, 'Season']
                value = player_df.loc[idx, col]
                mean = mean_df.loc[mean_df['Season'] == season, col].iloc[0]
                std = std_df.loc[std_df['Season'] == season, col].iloc[0]
                # scale so that 10 is 1 std away
                z_scores.loc[idx, col] = 100 + ((value - mean) / std * 10)

    z_scores['Season'] = player_df['Season']
    
    # cols to numeric
    for col in z_scores.columns:
        if col not in ['Season', 'MLBAMID']:
            z_scores[col] = pd.to_numeric(z_scores[col])
    
    # reinsert player names
    z_scores.insert(0, 'Name', player_df['Name']) 

    return z_scores

In [6]:
# df to store mean results
mean_values = pd.DataFrame(index=df['Season'].unique())

for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_mean = df.groupby('Season')[col].apply(mean_year)
        mean_values[col] = yearly_mean

# reset and rename index
mean_values = mean_values.reset_index()
mean_values = mean_values.rename(columns={'index': 'Season'})

# sort and display
mean_values = mean_values.sort_values(by=['Season'])
mean_values.head()

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
19,2002,473.684181,29.200233,0.088931,0.162881,0.294877,99.856949,0.221509,3.922156,-0.767819,1.888873,0.181982,0.703447,0.507012,0.860107,0.797336,0.094299,0.256508
22,2003,475.461939,29.191111,0.086575,0.159114,0.296402,99.927082,0.141683,4.016513,-0.608826,1.909976,0.221738,0.691527,0.527634,0.882055,0.800120,0.093138,0.260791
20,2004,477.651063,29.335336,0.087845,0.163399,0.299236,99.906874,0.255112,4.303784,-0.517574,1.945867,0.204546,0.693372,0.623672,0.855852,0.806547,0.088749,0.255794
14,2005,475.451649,29.259353,0.083354,0.159429,0.297611,100.116588,0.277356,3.656253,-0.893002,1.858494,0.214757,0.681891,0.550610,0.886206,0.813783,0.086017,0.255201
15,2006,483.988528,29.191618,0.085995,0.162801,0.303676,100.090682,0.168365,3.574442,-0.760455,1.862631,0.232215,0.671328,0.584224,0.889358,0.816041,0.084597,0.251676


In [7]:
# Create empty DataFrame to store results
std_values = pd.DataFrame(index=df['Season'].unique())

# Loop through numeric columns
for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_std = df.groupby('Season')[col].apply(calculate_weighted_std)
        std_values[col] = yearly_std

std_values = std_values.reset_index()
std_values = std_values.rename(columns={'index': 'Season'})

std_values = std_values.sort_values(by=['Season'])
std_values.head()

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
19,2002,184.243302,4.009692,0.037986,0.055977,0.035674,31.645660,3.121745,19.286650,9.129972,2.195603,0.047095,0.056035,0.108927,0.051809,0.061293,0.032167,0.028209
22,2003,185.106060,4.078477,0.035647,0.053696,0.034696,30.915634,3.282536,17.866265,9.428022,2.134635,0.048461,0.058644,0.107443,0.047224,0.060704,0.031931,0.027495
20,2004,189.820910,4.146485,0.037057,0.057978,0.036772,30.199233,3.575905,17.922925,9.712096,2.177108,0.051101,0.059314,0.106975,0.055553,0.063824,0.033339,0.030215
14,2005,182.884548,4.155987,0.032447,0.054885,0.035186,28.238546,3.328612,16.513000,10.060394,1.953790,0.057376,0.059317,0.117271,0.047543,0.061264,0.032130,0.027910
15,2006,184.138008,4.310582,0.035272,0.055416,0.036208,28.100105,3.600161,16.729148,9.906593,1.979241,0.060376,0.059520,0.112736,0.046223,0.061019,0.031551,0.028022


apply z scores

In [8]:
z_score_df = z_scores(player_df=df, mean_df=mean_values, std_df=std_values)
z_score_df.head()

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
0,brent clevlen,2007,74.826870,85.958971,74.686219,193.060161,107.344122,43.282571,98.653170,96.363387,100.657718,89.901996,138.703183,88.950287,71.666459,22.211544,37.187201,169.893575,204.596460
1,brett hayes,2009,74.297907,90.367791,72.475999,127.859623,108.219882,117.931506,99.866315,98.235778,100.843512,91.580480,170.668602,80.735141,60.421763,105.381033,65.578050,147.469352,199.428878
2,carlos peguero,2014,75.723755,96.257739,107.348483,147.612705,148.436984,92.738037,100.377615,97.711317,100.971051,91.256604,122.457976,73.452584,41.259125,91.336666,33.462464,156.578380,183.278087
3,seby zavala,2019,75.887131,92.057094,73.040518,181.646207,108.098997,44.146682,99.594613,95.874994,101.729855,90.245625,129.458257,111.272068,55.708080,34.476706,40.158323,179.134016,179.993300
4,franklin barreto,2020,77.834198,88.981943,76.647649,142.109822,78.737137,59.956124,101.610132,90.946521,99.887662,86.747334,120.942395,87.572409,55.233155,84.442687,56.517822,144.969583,165.941780


### xgboost model

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

make wrc+ value the next year

In [ ]:
# remove 2024 to train model bc we do not know the following year
train_z = z_score_df[z_score_df['Season'] < 2024].copy()
train_z['next_wrc+'] = np.nan # empty col


for idx in train_z.index:
    # retrive name, season
    name = train_z.loc[idx, 'Name']
    season = train_z.loc[idx, 'Season']
    # full df
    next_year_noscale = df[(df['Name'] == name) & (df['Season'] == season + 1)]
    next_year_scaled = z_score_df[(z_score_df['Name'] == name) & (z_score_df['Season'] == season + 1)]

    # add to df
    if not next_year_noscale.empty:
        train_z.loc[idx, 'next_wrc+'] = next_year_noscale['wRC+'].iloc[0]
        train_z.loc[idx, 'next_pa'] = next_year_scaled['PA'].iloc[0] # need for later anaylsis

# remove rows of nas (player retired or smth)
train_z = train_z.dropna(subset=['next_wrc+'])

# account for outliers
train_z = train_z.sort_values(by='next_pa')
train_z = train_z[500:] # account for pa outliers

# display
train_z.head()

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,...,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%,next_wrc+,next_pa
2348,james jones,2014,93.124187,91.150935,86.450058,100.915992,104.341247,89.177459,121.997733,95.019998,...,89.673440,104.069423,94.284934,96.900200,101.302897,99.125992,101.216814,112.683260,-17.659523,76.501035
243,abraham almonte,2022,77.741712,113.110026,81.630460,115.305251,116.951065,97.531712,101.504316,97.557596,...,91.170660,84.723319,81.715557,90.344380,93.094579,93.567247,98.092893,131.547944,-29.659769,76.512955
10568,brandon moss,2009,97.144742,90.367791,96.856278,104.149972,95.671776,91.519330,92.477105,88.434141,...,90.441236,106.613262,115.654087,91.248199,100.723735,95.070987,107.611132,96.124912,-0.427067,76.514364
5153,dan johnson,2007,100.840832,95.118336,116.984662,98.242247,84.803970,102.058830,94.976129,98.922836,...,95.886435,87.133795,81.067646,100.862952,102.836358,103.182040,91.889000,110.041557,85.840631,76.515082
7674,terrmel sledge,2004,98.332583,94.367915,100.496934,97.340803,97.300185,100.309734,100.425768,98.096189,...,99.163485,99.799235,91.606526,115.472546,98.010101,103.849664,94.507661,103.268192,90.051378,76.517882


dropped top and bottom 500 which may seem arb but it resulted in the wRC+ mean back at 100 and logically I like the look of the df a lot more. I am also making the choice of letter the model see the next year PA. I will build a seperate model to predict PA which I think is likely the most important step. without this it does not have context for injury. this is a important decison so I will experiemnt with it

In [11]:
X = train_z.drop(columns=['Name', 'Season', 'BsR', 'Off', 'Def', 'next_wrc+'])
Y = train_z['next_wrc+']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=26)

model

In [12]:
model = xgb.XGBRegressor(n_jobs=-1, n_estimators=1000, device = "cuda", eval_metric = ['mae', 'rmse']) 

##### hyper para tuning

random search to narrow space into grid search

In [13]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rnd_search_params = {
    'learning_rate': np.linspace(0.05, 0.2, 4),          
    'max_leaves': np.linspace(1, 20, 10, dtype=int),     
    'min_child_weight': np.linspace(1, 10, 10),         
    'subsample': np.linspace(0.3, 0.9, 7),               
    'colsample_bytree': np.linspace(0.4, 1, 7),          
    'early_stopping_rounds': np.linspace(10, 100, 10, dtype=int),  
    'max_depth': np.linspace(0, 20, 11, dtype=int)       
}

rnd_searcher = RandomizedSearchCV(model, rnd_search_params, random_state=26, n_iter=100, cv=10, n_jobs=-1, verbose=4)
rnd_searcher.fit(xtrain, ytrain, eval_set=[(xtest, ytest)])
print(rnd_searcher.best_params_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[0]	validation_0-mae:26.29414	validation_0-rmse:34.87658
[1]	validation_0-mae:25.86382	validation_0-rmse:34.34586
[2]	validation_0-mae:25.64799	validation_0-rmse:34.10080
[3]	validation_0-mae:25.46788	validation_0-rmse:33.88486
[4]	validation_0-mae:25.31679	validation_0-rmse:33.69554
[5]	validation_0-mae:25.29427	validation_0-rmse:33.66531
[6]	validation_0-mae:25.12667	validation_0-rmse:33.47196
[7]	validation_0-mae:24.97477	validation_0-rmse:33.30591
[8]	validation_0-mae:24.64465	validation_0-rmse:32.89981
[9]	validation_0-mae:24.51117	validation_0-rmse:32.74812
[10]	validation_0-mae:24.37899	validation_0-rmse:32.59856
[11]	validation_0-mae:24.08703	validation_0-rmse:32.24028
[12]	validation_0-mae:23.97465	validation_0-rmse:32.11488
[13]	validation_0-mae:23.88970	validation_0-rmse:32.00941
[14]	validation_0-mae:23.79563	validation_0-rmse:31.89053
[15]	validation_0-mae:23.72231	validation_0-rmse:31.80460
[16]	validation_0

c:\Users\dalto\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:09:09] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


[96]	validation_0-mae:20.48965	validation_0-rmse:27.94712
[97]	validation_0-mae:20.47556	validation_0-rmse:27.92605
[98]	validation_0-mae:20.46935	validation_0-rmse:27.91925
[99]	validation_0-mae:20.45521	validation_0-rmse:27.89997
[100]	validation_0-mae:20.44899	validation_0-rmse:27.89963
[101]	validation_0-mae:20.43757	validation_0-rmse:27.88780
[102]	validation_0-mae:20.41697	validation_0-rmse:27.87265
[103]	validation_0-mae:20.41233	validation_0-rmse:27.86574
[104]	validation_0-mae:20.40708	validation_0-rmse:27.86020
[105]	validation_0-mae:20.38857	validation_0-rmse:27.84421
[106]	validation_0-mae:20.37411	validation_0-rmse:27.82934
[107]	validation_0-mae:20.37102	validation_0-rmse:27.82455
[108]	validation_0-mae:20.36836	validation_0-rmse:27.81656
[109]	validation_0-mae:20.35988	validation_0-rmse:27.80926
[110]	validation_0-mae:20.35858	validation_0-rmse:27.80886
[111]	validation_0-mae:20.35298	validation_0-rmse:27.80228
[112]	validation_0-mae:20.35144	validation_0-rmse:27.80002
[

grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search_params = {
    'learning_rate': [0.05, 0.1],   
    'max_leaves': [10, 11, 12],
    'min_child_weight': [5, 6, 7],      
    'subsample': [0.5, 0.6, 0.7],            
    'colsample_bytree':[0.4, 0.5, 0.6],     
    'early_stopping_rounds':[40, 50, 60],
    'max_depth': [0, 2, 3]
}


grid_searcher = GridSearchCV(model, grid_search_params, cv=5, n_jobs=-1, verbose=4)
grid_searcher.fit(xtrain, ytrain, eval_set=[(xtest, ytest)])
print(grid_searcher.best_params_)

In [ ]:
opti_para = {
    'learning_rate': [0.05, 0.1],   
    'max_leaves': [10, 11, 12],
    'min_child_weight': [5, 6, 7],      
    'subsample': [0.5, 0.6, 0.7],            
    'colsample_bytree':[0.4, 0.5, 0.6],     
    'early_stopping_rounds':[40, 50, 60],
    'max_depth': [0, 2, 3]
}

xgb.cv for detailed analysis, optimal boosting rounds

In [ ]:
matrix_data = xgb.DMatrix(X, Y)

fmodel = xgb.cv(opti_para, num_boost_round=1000, nfold=10, metrics= ['mae', 'rmse'], as_pandas=True, seed=26)
fmodel.head()

##### results df

In [ ]:
results_df = xtest.copy()
results_df['Name'] = train_z.loc[xtest.index, 'Name']
results_df['Season'] = train_z.loc[xtest.index, 'Season']
results_df['next_pa'] = train_z.loc[xtest.index, 'next_pa']
results_df['pred_wrc+'] = fmodel.predict(xtest)
results_df['season_plus1_wrc+'] = ytest
results_df = results_df[['Name', 'Season', 'pred_wrc+', 'season_plus1_wrc+', 'next_pa']]
results_df['miss'] = (results_df['season_plus1_wrc+'] - results_df['pred_wrc+']).abs()
results_df.head()

,Name,Season,pred_wrc+,season_plus1_wrc+,next_pa,miss
11453,javier valentin,2006,78.294754,80.802973,88.504314,2.508219
12433,shea hillenbrand,2004,106.342834,108.601368,109.270786,2.258534
10194,michael martinez,2014,53.010242,67.626986,76.556207,14.616744
4931,jonathan herrera,2012,62.947689,74.699519,86.836808,11.751830
5643,zack collins,2021,83.327911,56.414984,81.707790,26.912928
